# Retrieve input for experimental design

## Import libraries and files

In [1]:
from openai import OpenAI
import openai
import os
import json
import requests
import nltk
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer


client = OpenAI()
openai.api_key = os.getenv("OPENAI_API_KEY")

with open('all_scores.json', 'r') as file:
    data = json.load(file)

## GPT functions to retrieve text and image based on all_scores.json

In [2]:
def generate_text_GPT(text_prompt):
    """
    Creates a new ChatGPT output, based on certain rules defined in the global variable "standard_rules".
    It clears the conversation history and add the new list items to it (starting a new chat).

    Input
    A text prompt that is meant for ChatGPT in string

    Returns
    ChatGPT output in a string

    """
    messages = [{"role": "system", "content": "Create the output without any markdown or markup"}]
    messages.append({"role": "user", "content": text_prompt})

    new = client.chat.completions.create(
                model="gpt-4o",
                messages=messages
            )

    # only get the output message 
    output = new.choices[0].message.content

    return output

def generate_image_GPT(text_prompt):
    """
    Creates a new DALLE-3 output, based on certain rules defined in the global variable "standard_rules".

    Input
    A text prompt that is meant for ChatGPT in string

    Returns
    Image URL (online for an hour)

    """

    response = client.images.generate(
                model="dall-e-3",
                prompt=text_prompt,
                size="1024x1024",
                quality="standard",
                n=1
            )

    # get image URL
    image_url = response.data[0].url

    return image_url


def output(data):
    amount_stages = data['stages']
    for i in range(amount_stages):
        i += 1
        print(f"log - stage: {i}")

        prompts = data[f'stage_{i}']['prompts']['text_prompt']
        for cat, prompt in prompts.items():
            print(f"log - category: {cat}, creating text")
            text_output = generate_text_GPT(prompt)
            data[f'stage_{i}']['output_text'][f'{cat}'] = text_output

            print('log - creating sentiment')
            blob = TextBlob(text_output, analyzer=NaiveBayesAnalyzer())
            data[f'stage_{i}']['sentiment'][f'{cat}'] = blob.sentiment

        prompts = data[f'stage_{i}']['prompts']['image_prompt']
        for cat, prompt in prompts.items():
            print(f"log - category: {cat}, creating image")
            image_url = generate_image_GPT(prompt)
            image_output = requests.get(image_url).content
            with open(f"images2/S{i}{cat}.png", "wb") as file:
                file.write(image_output)


In [134]:
# Run functions
output(data)

log - stage: 1
log - category: B, creating text
log - creating sentiment
log - category: PHA, creating text
log - creating sentiment
log - category: NHA, creating text
log - creating sentiment
log - category: PLA, creating text
log - creating sentiment
log - category: NLA, creating text
log - creating sentiment
log - category: B, creating image
log - category: PHA, creating image
log - category: NHA, creating image
log - category: PLA, creating image
log - category: NLA, creating image
log - stage: 2
log - category: B, creating text
log - creating sentiment
log - category: PHA, creating text
log - creating sentiment
log - category: NHA, creating text
log - creating sentiment
log - category: PLA, creating text
log - creating sentiment
log - category: NLA, creating text
log - creating sentiment
log - category: B, creating image
log - category: PHA, creating image
log - category: NHA, creating image
log - category: PLA, creating image
log - category: NLA, creating image
log - stage: 3
log

Testing

In [6]:
def check_openai_api_key(api_key):
    client = openai.OpenAI(api_key=api_key)
    try:
        client.models.list()
    except openai.AuthenticationError:
        return False
    else:
        return True
    
if check_openai_api_key(openai.api_key):
    print("Valid OpenAI API key.")
else:
    print("Invalid OpenAI API key.")

Valid OpenAI API key.


In [72]:
test_text = generate_text_GPT("""Create a detailed persona for the intended user of the chair, focusing on their lifestyle, preferences, and values relevant to seating. Describe key factors such as comfort needs, aesthetic tastes, and potential use environments. Additionally, create a visual portrait of this persona, capturing their characteristics and style preferences. Take into account my current emotional state without explicitly mentioning it. 
  Arousal: 0.8, Valence: 0.8. Include the emotion at the top.""")
print(test_text)

Cheerful

Persona Name: Alex Morgan

Lifestyle: Alex is a 35-year-old creative professional working in graphic design. They lead an active lifestyle, often balancing professional work with personal interests in art, outdoor activities, and social gatherings. Alex enjoys spending time at home, which serves as both a sanctuary for relaxation and a space to nurture creative ideas. Flexibility in living and work environments is essential for Alex, as they often transition between home office setup, studios, and occasional outdoor workspaces to draw inspiration from nature.

Comfort Needs: Comfort is a top priority for Alex, who often spends long hours working on projects. The chair must provide superior ergonomic support, helping maintain good posture to prevent strain during extended working sessions. Alex appreciates features like adjustable seat heights, lumbar support, and breathable materials, ensuring comfort throughout the day. For Alex, the ability to easily switch from a focused w

In [45]:
blob = TextBlob(test_text, analyzer=NaiveBayesAnalyzer())
blob.sentiment

Sentiment(classification='pos', p_pos=0.9999999993685619, p_neg=6.314555843163077e-10)

In [3]:
for i in range(0,2):
    image_url = generate_image_GPT(data['stage_2']['prompts']['all']['B']+ " Take into account my current emotional state without explicitly mentioning it, scales ranging from -1 to 1. Valence -1.0 arousal 1.")
    image_data = requests.get(image_url).content
    with open(f"HIGH{i}.png", "wb") as file:
        file.write(image_data)

In [135]:
with open('all_data.json', 'w') as f:
  f.write(json.dumps(data, indent=3))

In [140]:
with open('all_data.json', 'r') as f:
  data_ext = json.load(f)

with open('all_data copy.json', 'r') as f:
  data_simp = json.load(f)

In [144]:
print(data_ext['stage_1']['output_text']['PHA'])

Persona Description:

Name: Jamie Thompson

Age: 34

Occupation: Graphic Designer

Lifestyle: Jamie lives in a bustling urban environment. He splits his time between the home office and a co-working space downtown. An avid supporter of work-life balance, Jamie incorporates brief exercise routines and stretching into his daily schedule. He enjoys an active social life, frequently hosting friends at home for game nights and dinners. Weekends are reserved for outdoor activities like hiking or visiting art galleries.

Values and Preferences: 
- Comfort Needs: Jamie values ergonomic design in seating, prioritizing lumbar support and adjustable features. As someone who spends long hours at the computer, he requires a chair that supports a healthy posture to prevent fatigue.
- Aesthetic Tastes: With an eye for design, Jamie gravitates toward modern minimalist styles. He favors neutral color palettes with occasional bold accents, preferring materials that are both durable and sustainable. Slee

In [143]:
print(data_simp['stage_1']['output_text']['PHA'])

Persona Name: Emily Jacobs

Age: 34

Occupation: Freelance Graphic Designer

Lifestyle and Habits: Emily lives in an urban apartment with a mix of modern and vintage decor. She works primarily from home, often spending hours at her desk. She enjoys hosting friends for casual get-togethers, valuing moments of connection and comfort. Her weekends are filled with visits to art galleries, cozy cafes, and city parks. Emily leads an active lifestyle, practicing yoga and occasionally cycling around the city.

Seating Preferences: Comfort is paramount for Emily due to her long working hours, so ergonomic support is critical. She appreciates chairs with cushions that provide both support and softness. A high backrest is desirable for relaxation during breaks. She also values versatility in seating, preferring chairs that can easily transition from a workday setup to a comfortable spot for social gatherings.

Aesthetic Tastes: Emily has an eclectic taste, merging modern simplicity with a touch o